In [4]:
import torch
from torchvision import datasets, transforms

# Daten laden und präparieren

In diesem Beispiel sehen wir uns MNIST an.
Hierbei handelt es sich um eine Datenbank aus
70000 handgeschriebenen Ziffern, davon 60000 im Trainings-
und 10000 im Testdatensatz.

In [17]:
train_set = datasets.MNIST("data", train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(), torch.flatten]))
test_set = datasets.MNIST("data", train=False, download=True,
                       transform=transforms.Compose([transforms.ToTensor(), torch.flatten]))

In [18]:
train_set

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               <built-in method flatten of type object at 0x00007FFD5EC0A000>
           )

Diesen Datensatz packen wir nun in einen passenden `DataLoader`.
Mit diesem können wir gut über Datensätze iterieren.

In [19]:
batch_size = 50

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

Das erste Element des DataLoaders sieht wie folgt aus:

In [20]:
fst_data = next(iter(train_loader))
print(fst_data)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([9, 6, 0, 0, 7, 1, 5, 3, 2, 8, 7, 7, 4, 2, 6, 1, 4, 1, 1, 6, 2, 0, 6, 0,
        7, 2, 7, 0, 6, 0, 5, 2, 2, 4, 3, 5, 9, 7, 0, 4, 5, 3, 5, 6, 3, 5, 4, 8,
        7, 3])]


Betrachten beispielhaft wir die erste Ziffer.

In [22]:
print(fst_data)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([9, 6, 0, 0, 7, 1, 5, 3, 2, 8, 7, 7, 4, 2, 6, 1, 4, 1, 1, 6, 2, 0, 6, 0,
        7, 2, 7, 0, 6, 0, 5, 2, 2, 4, 3, 5, 9, 7, 0, 4, 5, 3, 5, 6, 3, 5, 4, 8,
        7, 3])]


In [21]:
fst_data

fst_image = fst_data[0][0]
fst_digit = fst_data[1][0].item()

import matplotlib.pyplot as plt
plt.imshow(fst_image.view(fst_image.shape[1], fst_image.shape[2]))
plt.title(f"Value: {fst_digit}")

IndexError: tuple index out of range

# Neuronales Netz erstellen

### Multilayer Perceptron
Nun erstellen wir die Architektur des neuronalen Netzes. Wir verwenden hier ein Fully Connected Neural Network mit einem Hidden Layer mit 64 Knoten.

<img src="assets/fcnn-28x28-64-10.svg"/>

In [23]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28*28, 64)
        self.fc2 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=1)
        return x

In [24]:
lr_rate = 0.01

In [25]:
model = MultiLayerPerceptron()
model

MultiLayerPerceptron(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

# Neuronales Netz trainieren

Jetzt initialisieren wir die Loss class und den Optimizer (Stochastic Gradient Descent).

In [26]:
loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [27]:
def calculate_accuracy(loader):
    correct = 0
    total = 0
    for images_test, labels_test in loader:
        images_test = Variable(images_test.view(-1, 28*28))
        outputs = model(images_test)
        _, predicted = torch.max(outputs.data, 1)
        total += labels_test.size(0)
        correct += (predicted == labels_test).sum()
    acc = correct.item() / total
    return acc

In [28]:
from torch.autograd import Variable

epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for images, labels in train_loader:
        # we use torch.autograd.Variable for backpropagation
        # images = Variable(images.view(-1, 28 * 28))
        images = Variable(images)
        labels = Variable(labels)

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    test_accuracy = calculate_accuracy(test_loader)
    print(f"Loss: {loss.item():.5f}. Test Accuracy: {test_accuracy:.3f}.")

Epoch 1/3
Loss: 2.27279. Test Accuracy: 0.307.
Epoch 2/3
Loss: 2.06779. Test Accuracy: 0.549.
Epoch 3/3
Loss: 1.85221. Test Accuracy: 0.659.
